In [27]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, SimpleRNN, Dropout
from sklearn.preprocessing import MinMaxScaler
import joblib  # For saving the scaler
import os
import pandas as pd

In [33]:
import numpy as np
import pickle
from sklearn.preprocessing import MinMaxScaler

def scale_and_save_data(X_train, y_train, X_val, y_val, X_test, y_test, output_file):
    """
    Scales the given stock price data (for Close option) using MinMaxScaler and saves the scaled data and scalers to a .pkl file.

    Parameters:
        X_train (numpy.ndarray): Training features.
        y_train (numpy.ndarray): Training labels (Close prices).
        X_val (numpy.ndarray): Validation features.
        y_val (numpy.ndarray): Validation labels (Close prices).
        X_test (numpy.ndarray): Testing features.
        y_test (numpy.ndarray): Testing labels (Close prices).
        output_file (str): Path to the .pkl file to save the data and scalers.
    """
    # Initialize scalers
    scaler_X = MinMaxScaler(feature_range=(0, 1))
    scaler_y = MinMaxScaler(feature_range=(0, 1))

    # Reshape y data to 2D for scaling (Close prices)
    y_train = y_train.reshape(-1, 1)
    y_val = y_val.reshape(-1, 1)
    y_test = y_test.reshape(-1, 1)

    # Fit and transform the scalers on the respective datasets
    X_train_scaled = scaler_X.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
    X_val_scaled = scaler_X.transform(X_val.reshape(-1, X_val.shape[-1])).reshape(X_val.shape)
    X_test_scaled = scaler_X.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

    y_train_scaled = scaler_y.fit_transform(y_train)
    y_val_scaled = scaler_y.transform(y_val)
    y_test_scaled = scaler_y.transform(y_test)

    # Save scaled data and scalers to a .pkl file
    data_to_save = {
        "X_train": X_train_scaled,
        "y_train": y_train_scaled,
        "X_val": X_val_scaled,
        "y_val": y_val_scaled,
        "X_test": X_test_scaled,
        "y_test": y_test_scaled,
        "scaler_X": scaler_X,
        "scaler_y": scaler_y,
    }

    with open(output_file, 'wb') as f:
        pickle.dump(data_to_save, f)

    print(f"Scaled stock price data (Close option) and scalers saved to {output_file}")

# Example usage
X_train, y_train, X_val, y_val, X_test, y_test = preprocess_stock_data(
    data_path_google, date_column='Date', close_column='Close', time_steps=60
)

output_file = "scaled_stock_data_close_option.pkl"
scale_and_save_data(X_train, y_train, X_val, y_val, X_test, y_test, output_file)



Scaled stock price data (Close option) and scalers saved to scaled_stock_data_close_option.pkl


In [34]:
class StockPricePredictor_Google:
    def __init__(self, input_shape, learning_rate=0.0007971184552975506, num_layers=2, units=256):
        self.input_shape = input_shape
        self.learning_rate = learning_rate
        self.num_layers = num_layers
        self.units = units
        self.model = self._build_model()

    def _build_model(self):
        model = Sequential()
        for i in range(self.num_layers):
            # Add RNN layers, making the last layer return sequences only if it's not the final RNN
            return_sequences = i < (self.num_layers - 1)
            model.add(SimpleRNN(self.units, activation='relu', return_sequences=return_sequences, input_shape=self.input_shape))
            model.add(Dropout(0.2))  # Regularization

        # Output layer for regression (single output: close value)
        model.add(Dense(1))

    # Compile the model with explicit loss and metric functions
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=self.learning_rate),
            loss=tf.keras.losses.MeanSquaredError(),  # Explicitly use the TensorFlow implementation
            metrics=[tf.keras.metrics.MeanAbsoluteError()]  # Use TensorFlow's metric function
        )
        return model

    def train(self, X_train, y_train, batch_size=16, epochs=50, validation_data=None):
        history = self.model.fit(X_train, y_train, 
                                 batch_size=batch_size, 
                                 epochs=epochs, 
                                 validation_data=validation_data,
                                 verbose=2)
        return history

    def predict(self, X):
        return self.model.predict(X)

    def save(self, model_path, scaler_path, scaler):
        # Save the model
        self.model.save(model_path)
        print(f"Model saved to {model_path}")

        # Save the scaler
        joblib.dump(scaler, scaler_path)
        print(f"Scaler saved to {scaler_path}")

    @staticmethod
    def load(model_path, scaler_path):
        # Load the model
        model = load_model(model_path)
        print(f"Model loaded from {model_path}")

        # Load the scaler
        scaler = joblib.load(scaler_path)
        print(f"Scaler loaded from {scaler_path}")

        return model, scaler




In [35]:
data_path_google = "inputs/google_stock_cleaned.csv"

X_train, y_train, X_val, y_val, X_test, y_test = scale_and_save_data(X_train, y_train, X_val, y_val, X_test, y_test, output_file)

Scaled stock price data (Close option) and scalers saved to scaled_stock_data_close_option.pkl


TypeError: cannot unpack non-iterable NoneType object

In [32]:
# Sample Usage
if __name__ == "__main__":
   
    # Scale data
    scaler = MinMaxScaler()
    y_train_scaled = scaler.fit_transform(y_train.reshape(-1, 1)).flatten()
    y_val_scaled = scaler.transform(y_val.reshape(-1, 1)).flatten()

    # Initialize and train the predictor
    predictor = StockPricePredictor_Google(input_shape=(X_train.shape[1], X_train.shape[2]))
    history = predictor.train(X_train, y_train_scaled, batch_size=16, epochs=20, validation_data=(X_val, y_val_scaled))

    # Save model and scaler
    model_path = "stock_price_rnn_model.h5"
    scaler_path = "stock_price_scaler.pkl"
    predictor.save(model_path, scaler_path, scaler)

    # Load model and scaler for inference
    loaded_model, loaded_scaler = StockPricePredictor.load(model_path, scaler_path)
    predictions = loaded_model.predict(X_val)
    predictions_rescaled = loaded_scaler.inverse_transform(predictions)

    print("Predictions (rescaled):", predictions_rescaled[:5])

Epoch 1/20


D:\stock_price_prediction\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


50/50 - 4s - 80ms/step - loss: 0.0960 - mean_absolute_error: 0.2612 - val_loss: 0.1009 - val_mean_absolute_error: 0.2749
Epoch 2/20
50/50 - 1s - 20ms/step - loss: 0.0895 - mean_absolute_error: 0.2522 - val_loss: 0.0943 - val_mean_absolute_error: 0.2671
Epoch 3/20
50/50 - 1s - 20ms/step - loss: 0.0861 - mean_absolute_error: 0.2494 - val_loss: 0.1064 - val_mean_absolute_error: 0.2773
Epoch 4/20
50/50 - 1s - 20ms/step - loss: 0.0853 - mean_absolute_error: 0.2471 - val_loss: 0.0934 - val_mean_absolute_error: 0.2680
Epoch 5/20
50/50 - 1s - 20ms/step - loss: 0.0844 - mean_absolute_error: 0.2475 - val_loss: 0.0922 - val_mean_absolute_error: 0.2661
Epoch 6/20
50/50 - 1s - 21ms/step - loss: 0.0819 - mean_absolute_error: 0.2444 - val_loss: 0.0925 - val_mean_absolute_error: 0.2660
Epoch 7/20
50/50 - 1s - 20ms/step - loss: 0.0827 - mean_absolute_error: 0.2458 - val_loss: 0.0933 - val_mean_absolute_error: 0.2660
Epoch 8/20
50/50 - 1s - 20ms/step - loss: 0.0805 - mean_absolute_error: 0.2402 - val_lo

Model saved to stock_price_rnn_model.h5
Scaler saved to stock_price_scaler.pkl
Model loaded from stock_price_rnn_model.h5
Scaler loaded from stock_price_scaler.pkl
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step
Predictions (rescaled): [[0.5850826 ]
 [0.56948256]
 [0.538114  ]
 [0.49103233]
 [0.3285896 ]]
